<a href="https://colab.research.google.com/github/93model/seoul_2030_night_life_pop/blob/main/seoul_2030_night_life_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 폰트 설정이 잘 안 된다면 해당 셀의 마지막 줄 주석을 풀고 실행해 보세요.
# colab 사용시에도 아래 함수를 활용해 보세요.
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())

In [ ]:
# Win
# plt.rc("font", family="Malgun Gothic")
# Mac
# plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

# 폰트가 선명하게 보이도록 설정

from IPython.display import set_matplotlib_formats

set_matplotlib_formats("retina")

1. 데이터 로드 


1.1 행정동별 8월 생활인구 평균(시간,날짜)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
life_pop_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/NP_1/LOCAL_PEOPLE_20220812.csv", encoding='cp949')
life_pop_df

In [ ]:
life_pop_df.columns

In [ ]:
life_pop_df_columns  = ['시간대구분', '행정동코드', '집계구코드', '총생활인구수',
                        '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
                        '여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수' ]

In [ ]:
life_pop_df = life_pop_df[life_pop_df_columns ]
life_pop_df

In [ ]:
life_pop_df_1 = life_pop_df.replace("*", np.NaN)
life_pop_df_1 

In [ ]:
life_pop_df_1.info()

In [ ]:
life_pop_df_1['남자20세부터24세생활인구수']= pd.to_numeric(life_pop_df_1['남자20세부터24세생활인구수'])

In [ ]:
life_pop_df_1[['시간대구분', '행정동코드', '집계구코드', '총생활인구수',
                        '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
                        '여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수' ]] =life_pop_df_1[['시간대구분',
                        '행정동코드', '집계구코드', '총생활인구수',
                        '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
                        '여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수' ]].apply(pd.to_numeric)

In [ ]:
life_pop_df_2 = life_pop_df_1.dropna()
life_pop_df_2 

In [ ]:
Time_con = (life_pop_df_2.시간대구분 <= 5) | (life_pop_df_2.시간대구분 >= 19) # 조건식 작성
life_pop_df_3= life_pop_df_2[Time_con]
life_pop_df_3 

In [ ]:
life_pop_df_3['2030_men'] = life_pop_df_3['남자20세부터24세생활인구수'] + life_pop_df_3['남자25세부터29세생활인구수'] + life_pop_df_3['남자30세부터34세생활인구수']  + life_pop_df_3['남자35세부터39세생활인구수'] 
life_pop_df_3['2030_women'] = life_pop_df_3['여자20세부터24세생활인구수'] + life_pop_df_3['여자25세부터29세생활인구수'] + life_pop_df_3['여자30세부터34세생활인구수']  + life_pop_df_3['여자35세부터39세생활인구수'] 
life_pop_df_3


In [ ]:
life_pop_df_men =pd.pivot_table(life_pop_df_3, index = ['행정동코드'],values = '2030_men',aggfunc = 'sum')
life_pop_df_men.reset_index(inplace=True)
life_pop_df_men

In [ ]:
life_pop_df_women =pd.pivot_table(life_pop_df_3, index = ['행정동코드'],values = '2030_women',aggfunc = 'sum')
life_pop_df_women.reset_index(inplace=True)
life_pop_df_women

In [ ]:
life_pop_df_people =pd.pivot_table(life_pop_df_3, index = ['행정동코드'],values = '총생활인구수',aggfunc = 'sum')
life_pop_df_people.reset_index(inplace=True)
life_pop_df_people

In [ ]:
life_pop_dong_all=pd.merge(left = life_pop_df_men , right = life_pop_df_women, how = "left", on = "행정동코드")
life_pop_dong_all_1= pd.merge(left = life_pop_dong_all , right = life_pop_df_people, how = "left", on = "행정동코드")
life_pop_dong_all_1

In [ ]:
life_pop_dong_all_1['2030_poeple'] = life_pop_dong_all_1['2030_men'] + life_pop_dong_all_1['2030_women']
life_pop_dong_all_1

In [ ]:
dong_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Cp1_1/행정동코드_매핑정보_2018_.csv")
dong_df

In [ ]:
life_pop_dong_last=pd.merge(left = life_pop_dong_all_1 , right = dong_df, how = "left", on = "행정동코드")
life_pop_dong_last.columns=['dong_code','2030men','2030women','all_people','2030_people','name']
life_pop_dong_last

In [ ]:
life_pop_dong_last

In [ ]:
life_pop_dong_last['2030_chai'] = life_pop_dong_last['2030men'] - life_pop_dong_last['2030women']

In [ ]:
life_pop_dong_last

In [ ]:
life_pop_dong_last.to_csv('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/NP_1/life_pop_dong_1.csv')